# Create Merged Dataset

In this workbook, you will read in the `trip` and `fare` files. You are welcome to use DataFrame and/or SparkSQL API as you desire as long as it produces the expected results.

## Instructions:

1. Join both datasets such that you get a merged dataset with 21 unique fields. You need to determine how to join the dataset.
2. Once you create the merged dataset, you need to convert fields to the following types, since all fields were read is as string:
    * pickup_datetime and dropoff_datetime must be TIMESTAMP
    * passenger_count and rate_code must be INT
    * all other numeric fields must be FLOAT
    * the remaining fields stay as STRING
3. Save your merged and converted dataset to your own S3 bucket in parquet format.

You are welcome to add as many cells as you need below up until the next section. **You must include comments in your code.**

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkml").getOrCreate()

In [2]:
spark

In [ ]:
#load trip file into a DataFrame
trip_df = spark.read\
  .format('csv')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load('s3://bigdatateaching/nyctaxi-2013/parquet/trip')

In [ ]:
trip_df.printSchema()

In [ ]:
#load fare file into a DataFrame
fare_df = spark.read\
  .format('csv')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load('s3://bigdatateaching/nyctaxi-2013/parquet/fare')

In [ ]:
fare_df.printSchema()

In [ ]:
# Inner Join trip and fare dataframe on columns 'medallion', ' hack_license' and 'vendor_id'
merged_df=trip_df.join(fare_df, (trip_df.medallion == fare_df.medallion)
                    & (trip_df.hack_license == fare_df.hack_license)
                    & (trip_df.vendor_id == fare_df.vendor_id)).show()

In [ ]:
merged_df.printSchema()

In [ ]:
# convert pickup_datetime and dropoff_datetime from string to TIMESTAMP
from pyspark.sql.functions import unix_timestamp

merged_df= merged_df.withColumn('pickup_datetime',
                                unix_timestamp(col("pickup_datetime"), "yyyy-MM-dd hh:mm:ss").cast("timestamp"))
merged_df= merged_df.withColumn("dropoff_datetime", 
                                unix_timestamp(col("dropoff_datetime"),"yyyy-MM-dd HH:mm:ss").cast("timestamp"))

In [ ]:
# convert passenger_count and rate_code from string to INT
from pyspark.sql.types import IntegerType
merged_df = merged_df.withColumn("passenger_count", merged_df["passenger_count"].cast(IntegerType()))
merged_df = merged_df.withColumn("rate_code", merged_df["rate_code"].cast(IntegerType()))

In [ ]:
# convert all other numeric fields from string to float
for col_name in cols:
    df = df.withColumn(col_name, col(col_name).cast('float')).cache()

## In the following cells, please provide the requested code and output. Do not change the order and/or structure of the cells.

In the next cell, provide the code that saves your merged dataset to your S3 bucket.

In [ ]:
# Save your merged and converted dataset to your own S3 bucket in parquet format
import pyarrow.parquet as pq
pq.write_table(merged_df, 's3://bigdata6278exercise/merged.parquet')

In the next cell, print the schema of your merged dataset.

In [ ]:
merged_df.printSchema()

In the next cell, print the first 10 records of your merged dataset.

In [ ]:
merged_df.show(10)

In the next cell, print the row count of your merged dataset.

In [ ]:
merged_df.count()